# Fine-Tune *toxic-bert* with Reinforcement Learning (PPO) and PEFT to Generate Less-Toxic Summaries for Azerbaijani Langueges

In this notebook, you will fine-tune a *toxic-bert* model to generate less toxic content with Meta AI's hate speech reward model. The reward model is a binary classifier that predicts either "not hate" or "hate" for the given text. You will use Proximal Policy Optimization (PPO) to fine-tune and reduce the model's toxicity.

# Table of Contents

- [ 1 - Set up Required Dependencies](#1)
- [ 2 - Load toxic-bert Model, Prepare Reward Model and Toxicity Evaluator](#2)
  - [ 2.1 - Load Data and toxic-bert Model Fine-Tuned with Summarization Instruction](#2.1)
  - [ 2.2 - Prepare Reward Model](#2.2)
  - [ 2.3 - Evaluate Toxicity](#2.3)
- [ 3 - Perform Fine-Tuning to Detoxify the Summaries](#3)
  - [ 3.1 - Initialize `PPOTrainer`](#3.1)
  - [ 3.2 - Fine-Tune the Model](#3.2)
  - [ 3.3 - Evaluate the Model Quantitatively](#3.3)
  - [ 3.4 - Evaluate the Model Qualitatively](#3.4)

<a name='1'></a>
## 1 - Set up Required Dependencies

Install the required packages to use PyTorch and Hugging Face transformers and datasets.


In [5]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 --quiet

# Installing the Reinforcement Learning library directly from github.
%pip install git+https://github.com/lvwerra/trl.git@25fa1bd    

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Ignored the following yanked versions: 0.3.0a0
ERROR: Could not find a version that satisfies the requirement torchdata==0.5.1 (from versions: 0.3.0a1, 0.3.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.8.0, 0.9.0, 0.10.0, 0.10.1)
ERROR: No matching distribution found for torchdata==0.5.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/lvwerra/trl.git (to revision 25fa1bd) to /tmp/pip-req-build-6gh671yq
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-6gh671yq
  Running command git checkout -q 25fa1bd
  Resolved https://github.com/lvwerra/trl.git to commit 25fa1bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... d

Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook. 

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

<a name='2'></a>
## 2 - Load *toxic-bert* Model, Prepare Reward Model and Toxicity Evaluator

<a name='2.1'></a>
### 2.1 - Load Data and *toxic-bert* Model Fine-Tuned 

You will keep working with the same Hugging Face dataset [En-Az](https://huggingface.co/datasets/Zarifa/English-To-Azerbaijani) and the pre-trained model [toxic-bert](https://huggingface.co/unitary/toxic-bert). 

In [8]:
huggingface_dataset_name = "Zarifa/English-To-Azerbaijani"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 5161
    })
})

In [9]:
# Specify the model name
model_name = "unitary/toxic-bert"

# Load the model
original_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/omniaz/Desktop/jobs/2025/My-Daily-Python-Learning/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The next step will be to preprocess the dataset. I will take only a part of it, then filter the translation of a particular length (just to make those examples long enough and, at the same time, easy to read). Then wrap each sentences with the instruction and tokenize the prompts. Save the token ids in the field `input_ids` and decoded version of the prompts in the field `query`.

You could do that all step by step in the cell below, but it is a good habit to organize that all in a function `build_dataset`:

In [10]:
def tokenize_function(example):
    start_prompt = 'Translate the following conversation.\n\n'
    end_prompt = '\n\nTranslate: '
    prompt = [start_prompt + sentence + end_prompt for sentence in [ex['aze'] for ex in example['translation']]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer([ex['en'] for ex in example['translation']], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map: 100%|██████████| 5161/5161 [00:03<00:00, 1616.26 examples/s]


In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(['id'])

In [12]:
tokenized_datasets_training = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_datasets_validation = tokenized_datasets.filter(lambda example, index: index % 1001 == 0, with_indices=True)

Filter: 100%|██████████| 5161/5161 [00:01<00:00, 3085.17 examples/s]


Prepare a function to pull out the number of model parameters (it is the same as in the Fine Tune LLM for MT):

In [13]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

Add the adapter to the original *toxic-bert* model. I need to pass them to the constructed PEFT model, also putting `is_trainable=True`.

In [ ]:
# Configure LoRA for the Helsinki model
# Dynamically generate the target modules
encoder_layers = [f"model.encoder.layers.{i}.self_attn.{proj}" for i in range(6) for proj in ["k_proj", "v_proj", "q_proj"]]
decoder_self_attn_layers = [f"model.decoder.layers.{i}.self_attn.{proj}" for i in range(6) for proj in ["k_proj", "v_proj", "q_proj"]]
decoder_cross_attn_layers = [f"model.decoder.layers.{i}.encoder_attn.{proj}" for i in range(6) for proj in ["k_proj", "v_proj", "q_proj"]]

# Combine all target modules
target_modules = encoder_layers + decoder_self_attn_layers + decoder_cross_attn_layers

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=target_modules,  # Explicitly specified target modules
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",  # No bias reparameterization
    task_type=TaskType.SEQ_2_SEQ_LM  # Sequence-to-sequence task
)


peft_model = PeftModel.from_pretrained(original_model, 
                                       'peft-sentence-translate-training-1736885736/', 
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16, 
                                       device_map="auto",                                       
                                       is_trainable=True)

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')


In this part, I am preparing to fine-tune the LLM using Reinforcement Learning (RL). RL will be briefly discussed in the next section of this part, but at this stage, I just need to prepare the Proximal Policy Optimization (PPO) model passing the instruct-fine-tuned PEFT model to it. PPO will be used to optimize the RL policy against the reward model.

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,                                                               
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

During PPO, only a few parameters will be updated. Specifically, the parameters of the `ValueHead`. More information about this class of models can be found in the [documentation](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model). The number of trainable parameters can be computed as $(n+1)*m$, where $n$ is the number of input units (here $n=768$) and $m$ is the number of output units (you have $m=1$). The $+1$ term in the equation takes into account the bias term.

Now create a frozen copy of the PPO which will not be fine-tuned - a reference model. The reference model will represent the LLM before detoxification. None of the parameters of the reference model will be updated during PPO training. This is on purpose.

In [ ]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Everything is set. It is time to prepare the reward model!

toxic-bert